In [80]:
import os
os.chdir("..")

In [81]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [82]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [85]:
import bs4 
import html

def remove_tags(message):
    parsed = bs4.BeautifulSoup(message, "html.parser").text
    print(parsed)
    return bs4.BeautifulSoup(message, "html.parser").text


In [71]:

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=20&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

In [72]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))
    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [ ]:
for song in data['tracks']:
    print(song)
    id = str(song['artists'][0]['id'])
    song_id = song['id']
    print(song_id)
    if Artist.objects.filter(api_id=id).exists():
        print('Developer msg: data already exists in database')
        continue
    else:
        conn.request("GET", "/artists/?ids={id}".format(
                        id=id), headers=headers)
        res = conn.getresponse()
        datas = json.loads(res.read().decode('utf-8'))['artists'][0]
        artist = Artist(api_id=id, name = datas['name'], popularity=datas['popularity'], image=datas['images'][0]['url'], followers=datas['followers']['total'])
        artist.save()
        song_id = song['id']
        this_song = Song.objects.get(api_id=song_id)
        print(this_song)
        artist.songs.add(this_song)
        artist.save()



In [83]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

In [86]:
data = Artist.objects.all()

for i in data:
    id = str(i)
    print(i)
    conn.request("GET", "/artist_overview/?id={id}".format(
                    id=id), headers=headers)
    res = conn.getresponse()
    datas = json.loads(res.read().decode('utf-8'))
    print(datas)
    update = Artist.objects.get(api_id=id)
    text = html.unescape(datas['data']['artist']['profile']['biography']['text'])
    update.overview = remove_tags(text)
    update.save()
    print(text)

2AfmfGFbe0A0WsTYm0SDTx
{'data': {'artist': {'id': '2AfmfGFbe0A0WsTYm0SDTx', 'uri': 'spotify:artist:2AfmfGFbe0A0WsTYm0SDTx', 'following': False, 'sharingInfo': {'shareUrl': 'https://open.spotify.com/artist/2AfmfGFbe0A0WsTYm0SDTx?si=GUBHyZ9sQLei1Wy9ETn5tQ', 'shareId': 'GUBHyZ9sQLei1Wy9ETn5tQ'}, 'profile': {'name': '(G)I-DLE', 'verified': True, 'pinnedItem': None, 'biography': {'text': 'The &#34;G&#34; in (G)I-DLE is a short for Girl, &#34;I&#34; means I as in myself, and &#34;DLE&#34; (들) is a Korean compound word used to form plurals. (G)I-DLE is a global quintet who construct their own musical universe with their exceptional concepts and songwriting skills. The five members of the group are MIYEON, MINNIE, SOYEON, YUQI, and SHUHUA.'}, 'externalLinks': {'items': [{'name': 'FACEBOOK', 'url': 'https://facebook.com/G.I.DLE.CUBE'}, {'name': 'INSTAGRAM', 'url': 'https://instagram.com/official_g_i_dle'}, {'name': 'TWITTER', 'url': 'https://twitter.com/G_I_DLE'}]}, 'playlists': {'totalCount': 